<a href="https://colab.research.google.com/github/traderjoevitamins/librarian/blob/main/librarian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [ ]:
!pip install --upgrade 'tensorflow_data_validation[visualization]<2'
!pip install -U scikit-learn
!pip install tensorflow-text
!pip install bert-tensorflow
!pip install bert-for-tf2
!pip install transformers


In [23]:
!pip install torch

In [24]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_data_validation as tfdv
import pandas as pd
import nltk
import tensorflow_hub as hub
import nltk
import sklearn
import bert
import math
import torch

from transformers import BertTokenizer, BertModel
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from bert import tokenization
from tokenizers import Tokenizer
from tokenizers.models import BPE
from transformers import AutoTokenizer

#from bert.tokenization import FullTokenizer
#FullTokenizer = bert.bert_tokenization.FullTokenizer


nltk.download('punkt')
nltk.download('punkt')
print('TF version:', tf.__version__)
print('NLKT version:', nltk.__version__)
print('Hub version:', hub.__version__)
print('TFDV version:', tfdv.version.__version__)


TF version: 2.13.0
NLKT version: 3.8.1
Hub version: 0.14.0
TFDV version: 1.14.0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
goodreads = pd.read_csv('/content/finalgoodreads.csv')
goodreads['enjoyed'] = 1
goodreads = goodreads[['Title', 'Author', 'ISBN13', 'synopsis', 'enjoyed']]
nyt1 = pd.read_csv('/content/nyt1.csv')
nyt2 = pd.read_csv('/content/nyt2.csv')
goodreads_synopsis = goodreads['synopsis']
nyt = pd.concat([nyt1, nyt2], axis = 0)
nyt_synopsis = nyt['synopsis']

goodreads_synopsis = goodreads_synopsis.str.replace('<p class="null1">','').replace('</p>\n<p>', '')
goodreads_synopsis = goodreads_synopsis.str.replace('<p>', '').replace('<', '').replace('>', '')
goodreads_synopsis = goodreads_synopsis.str.replace('<BR>', '').replace('</p><h3>', '').replace('>', '')
goodreads_synopsis = str(goodreads_synopsis)


nyt_synopsis = nyt_synopsis.str.replace('<p class="null1">','').replace('</p>\n<p>', '')
nyt_synopsis = nyt_synopsis.str.replace('<p>', '').replace('<', '').replace('>', '')
nyt_synopsis = nyt_synopsis.str.replace('<BR>', '').replace('</p><h3>', '').replace('>', '')
nyt_synopsis = str(nyt_synopsis)



In [20]:
#tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True) # Making the model return all hidden-states.
model.eval()


In [31]:
gtokenized_text = []
gsegments_ids = []
gtokens_tensor = []
gsegments_tensors = []

In [32]:
for i, syno in enumerate(goodreads_synopsis):
 gtokenized_text.append(tokenizer.encode(syno, add_special_tokens=True))
 gsegments_ids.append([1] * len(tokenized_text[i]))
 gtokens_tensor.append(torch.tensor([tokenized_text[i]]))
 gsegments_tensors.append(torch.tensor([segments_ids[i]]))

In [45]:
outputs = []
hidden_states = []
with torch.no_grad():
 for i in range(len(gtokens_tensor)):
  outputs.append(model(gtokens_tensor[i], gsegments_tensors[i]))
 # we set `output_hidden_states = True`, the third item will be the
 # hidden states from all layers. See the documentation for more details:
 # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
 hidden_states.append(outputs[i][2])


AttributeError: ignored

In [10]:
max_seq_length = len(nyt_synopsis)  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

gtokens = tokenizer.tokenize(goodreads_synopsis)
ntokens = tokenizer.tokenize(nyt_synopsis)

gtokens = ["[CLS]"] + gtokens + ["[SEP]"]
ntokens = ["[CLS]"] + ntokens + ["[SEP]"]

ginput_ids = get_ids(gtokens, tokenizer, max_seq_length)
ninput_ids = get_ids(ntokens, tokenizer, max_seq_length)

ginput_masks = get_masks(gtokens, max_seq_length)
ninput_masks = get_masks(ntokens, max_seq_length)

ginput_segments = get_segments(gtokens, max_seq_length)
ninput_segments = get_segments(ntokens, max_seq_length)

pool_embs, all_embs = model.predict([[ginput_ids],[ginput_masks],[ginput_segments]])
#pool_embs, all_embs = model.predict([[ginput_ids],[ginput_masks],[ginput_segments]])

